In [ ]:
from chainer.datasets import LabeledImageDataset
train = LabeledImageDataset('./small_data/train/label.txt', './small_data/train/image')
from chainercv.transforms import resize
from chainer.datasets import TransformDataset

def transform(in_data):
    # データのリサイズ
    img, label = in_data
    img = resize(img, (224, 224))
    return img, label

train_val = TransformDataset(train, transform)

In [ ]:
from chainer.datasets import split_dataset_random
train, valid = split_dataset_random(train_val, int(len(train_val)*0.8), seed=0)

In [ ]:
from chainer import iterators

batchsize = 32 # クラスが多いときは多くする

# 
train_iter = iterators.SerialIterator(train, batchsize)
valid_iter = iterators.SerialIterator(
    valid, batchsize, repeat=False, shuffle=False)
# test_iter = iterators.SerialIterator(
#     test, batchsize, repeat=False, shuffle=False)

In [ ]:
import chainer.links as L
import chainer.functions as F

In [ ]:
import random
import numpy
import chainer

def reset_seed(seed=0):
    random.seed(seed)
    numpy.random.seed(seed)
    if chainer.cuda.available:
        chainer.cuda.cupy.random.seed(seed)

reset_seed(0)

In [42]:
class Mynet(chainer.Chain):

    def __init__(self, class_labels=2):
        super(Mynet, self).__init__()
        with self.init_scope():
            self.conv1_1 = L.Convolution2D(None, 16, ksize=5, pad=2, nobias=True)
            self.conv1_2 = L.Convolution2D(None, 16, ksize=5, pad=2, nobias=True)
            self.conv2_1 = L.Convolution2D(None, 32, ksize=3, pad=1, nobias=True)
            self.conv2_2 = L.Convolution2D(None, 32, ksize=3, pad=1, nobias=True)
            self.fc1 = L.Linear(None, 512, nobias=True)
            self.fc2 = L.Linear(None, class_labels, nobias=True)      

    def __call__(self, x):
        conv1_1 = self.conv1_1(x)
        conv1_1 = F.relu(conv1_1)
        conv1_2 = self.conv1_2(conv1_1)
        conv1_2 = F.relu(conv1_2)
        pool1 = F.max_pooling_2d(conv1_2, ksize=2, stride=2)
        conv2_1 = self.conv2_1(pool1)
        conv2_1 = F.relu(conv2_1)
        conv2_2 = self.conv2_2(conv2_1)
        conv2_2 = F.relu(conv2_2)
        pool2 = F.max_pooling_2d(conv2_2, ksize=2, stride=2)
        fc1 = self.fc1(pool2)
        fc1 = F.relu(fc1)
        fc2 = self.fc2(fc1)
        return fc2

gpu_id = -1  # CPUを用いる場合は、この値を-1にしてください

net = Mynet()

if gpu_id >= 0:
    net.to_gpu(gpu_id)

In [43]:
from chainer import optimizers
# オプティマイザ
optimizer = optimizers.SGD(lr=0.01).setup(net)

In [ ]:
import numpy as np
from chainer.dataset import concat_examples
from chainer.cuda import to_cpu

max_epoch = 100

while train_iter.epoch < max_epoch:

    # ---------- 学習の1イテレーション ----------
    train_batch = train_iter.next()
    x, t = concat_examples(train_batch, gpu_id)

    # 予測値の計算
    y = net(x)

    # ロスの計算
    loss = F.softmax_cross_entropy(y, t)

    # 勾配の計算
    net.cleargrads()
    loss.backward()

    # パラメータの更新
    optimizer.update()
    # --------------- ここまで ----------------

    # 1エポック終了ごとにValidationデータに対する予測精度を測って、
    # モデルの汎化性能が向上していることをチェックしよう
    if train_iter.is_new_epoch:  # 1 epochが終わったら

        # ロスの表示
        print('epoch:{:02d} train_loss:{:.04f} '.format(
            train_iter.epoch, float(to_cpu(loss.data))), end='')

        valid_losses = []
        valid_accuracies = []
        while True:
            valid_batch = valid_iter.next()
            x_valid, t_valid = concat_examples(valid_batch, gpu_id)

            # Validationデータをforward
            with chainer.using_config('train', False), \
                    chainer.using_config('enable_backprop', False):
                y_valid = net(x_valid)

            # ロスを計算
            loss_valid = F.softmax_cross_entropy(y_valid, t_valid)
            valid_losses.append(to_cpu(loss_valid.array))

            # 精度を計算
            accuracy = F.accuracy(y_valid, t_valid)
            accuracy.to_cpu()
            valid_accuracies.append(accuracy.array)

            if valid_iter.is_new_epoch:
                valid_iter.reset()
                break

        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(
            np.mean(valid_losses), np.mean(valid_accuracies)))



/anaconda/lib/python3.6/site-packages/chainer/functions/activation/log_softmax.py:18: RuntimeWarning: invalid value encountered in subtract
  y = x - m
/anaconda/lib/python3.6/site-packages/chainer/functions/activation/relu.py:40: RuntimeWarning: invalid value encountered in maximum
  return utils.force_array(numpy.maximum(x, 0, dtype=x.dtype)),
/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)
/anaconda/lib/python3.6/site-packages/chainer/functions/activation/relu.py:99: RuntimeWarning: invalid value encountered in greater
  y = (self.b > 0) * inputs[0]


epoch:01 train_loss:nan val_loss:nan val_accuracy:0.5000
epoch:02 train_loss:nan val_loss:nan val_accuracy:0.5000
epoch:03 train_loss:nan val_loss:nan val_accuracy:0.5000
epoch:04 train_loss:nan val_loss:nan val_accuracy:0.5000
epoch:05 train_loss:nan val_loss:nan val_accuracy:0.5000
epoch:06 train_loss:nan val_loss:nan val_accuracy:0.5000
epoch:07 train_loss:nan val_loss:nan val_accuracy:0.5000
epoch:08 train_loss:nan val_loss:nan val_accuracy:0.5000


In [ ]:
# テストデータでの評価
test_accuracies = []
while True:
    test_batch = test_iter.next()
    x_test, t_test = concat_examples(test_batch, gpu_id)

    # テストデータをforward
    with chainer.using_config('train', False), \
            chainer.using_config('enable_backprop', False):
        y_test = net(x_test)

    # 精度を計算
    accuracy = F.accuracy(y_test, t_test)
    accuracy.to_cpu()
    test_accuracies.append(accuracy.array)

    if test_iter.is_new_epoch:
        test_iter.reset()
        break

print('test_accuracy:{:.04f}'.format(np.mean(test_accuracies)))